## 抓取富邦人壽問答集

In [3]:
import requests
res = requests.get('https://www.fubon.com/life/direct/Faq.html')
res.encoding

'ISO-8859-1'

In [5]:
res.encoding = 'utf-8'
#res.text

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, 'lxml')

In [11]:
questions = [q.text for q  in soup.select('.faq__q')]
#questions

In [15]:
answers = [q.text.strip() for q  in soup.select('.faq__a')]

In [19]:
qa_ary = []
for q, a in zip(questions, answers):
    qa_ary.append({'questions':q,'answer':a})

In [21]:
import pandas
qa_df = pandas.DataFrame(qa_ary)
qa_df.to_excel('fubonqa.xlsx')

## 產生一個自動問答機器人

In [22]:
import pandas
fubonqa = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/fubonpy/master/data/fubonqa.xlsx')

In [40]:
import jieba

q = '請問要保人是什麼?'

corpus    = [' '.join(jieba.cut(q))]
questions = [q]
answers   = ['DUMMY']
for rec in fubonqa.iterrows():
    corpus.append(' '.join(jieba.cut(rec[1].questions)))
    questions.append(rec[1].questions)
    answers.append(rec[1].answer)

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
X

<31x114 sparse matrix of type '<class 'numpy.int64'>'
	with 183 stored elements in Compressed Sparse Row format>

In [42]:
from sklearn.metrics.pairwise import cosine_similarity
cs = cosine_similarity(X[0], X)
pos = cs[0].argsort()[::-1][1]
questions[pos]
answers[pos]

'要保人是指對保險標的具有保險利益，向保險人申請訂立保險契約，並負有交付保險費義務之人。'

In [51]:
import jieba

q = '請問要保人是什麼?'

def autoQA(q,threshold):
    corpus    = [' '.join(jieba.cut(q))]
    questions = [q]
    answers   = ['DUMMY']
    for rec in fubonqa.iterrows():
        corpus.append(' '.join(jieba.cut(rec[1].questions)))
        questions.append(rec[1].questions)
        answers.append(rec[1].answer)
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    cs = cosine_similarity(X[0], X)
    pos = cs[0].argsort()[::-1][1]
    if cs[0][pos] > threshold:
        return answers[pos]
    else:
        return '我現在還在學習您的問題, 等我變聰明以後我就可以回答您了'

In [53]:
autoQA('請問要保人是什麼?', 0.5)

'要保人是指對保險標的具有保險利益，向保險人申請訂立保險契約，並負有交付保險費義務之人。'

In [54]:
autoQA('請問今天天氣好嗎', 0.5)

'我現在還在學習您的問題, 等我變聰明以後我就可以回答您了'

## 產生一個可對話機器人

In [58]:
import speech_recognition as sr
def listenToMe():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("請說出您的問題!")
        audio = r.listen(source)
    return r.recognize_google(audio, language='zh-TW')

In [59]:
listenToMe()

請說出您的問題!


'請問什麼是要保人'

In [60]:

from gtts import gTTS
from pygame import mixer
mixer.init()

import tempfile
def speak(sentence):
    with tempfile.NamedTemporaryFile(delete=True) as fp:
        tts = gTTS(text=sentence, lang='zh')
        tts.save("{}.mp3".format(fp.name))
        mixer.music.load('{}.mp3'.format(fp.name))
        mixer.music.play()
speak('大家好')

In [67]:
speak(autoQA(listenToMe(), 0.5))

請說出您的問題!


## 新聞分群

In [78]:
import requests
from bs4 import BeautifulSoup
res = requests.get('https://raw.githubusercontent.com/ywchiu/fuboni/master/data/1435449602.xml')

In [97]:
from xml.dom import minidom
from xml.etree import ElementTree
import jieba.analyse
import re

events=ElementTree.fromstring(res.text)

corpus = []
titles = []
for elem in events.findall('./channel/item'):
    title       = elem.find('title').text
    description = elem.find('description').text
    ## original
    #corpus.append(' '.join(jieba.cut(description)))
    
    ## filter out non-chinese word
    #ary = []
    #for w in jieba.cut(description):
    #    m = re.match('^[\u4e00-\u9fa5]+$', w)
    #    if m:
    #        ary.append(w)
    #corpus.append(' '.join(ary))
    
    ## recommend method
    corpus.append(' '.join([w for w in jieba.cut(description) \
                            if re.match('^[\u4e00-\u9fa5]+$', w)]))
    titles.append(title)

In [98]:
corpus[0]

'新北市 八仙 水上 樂園 昨晚 發 生粉 塵 爆炸 新北市 衛生局 統計 到 目前 為止 由 救護車 送 醫再 加上 自行 送醫 的 人數 已經 有 人 其中 人 重傷 而 新北市 長 朱立倫 昨晚 到 現場 坐鎮 指揮 了解 狀況 並馬上 要求 追究 責任 徹查 並要 八仙 樂園 立刻 關園 停業 八仙 樂園 業者 雖然 表示 活動 是 外包 的 但 也 說 會 配合 暫時 停止 營業 接受 調查 警方 今天 凌晨 也 帶 回 五名 相關 人 漏夜 偵訊 包括 現場 呂姓 活動 負責人 邱姓 硬體 設備 人員 和 廖姓 特效 人員 以及 兩名 在 現場 噴灑 彩色 粉塵 的 名 工作 人員 訊後將 依業務 過失 重大 傷害 與 公共 危險 罪嫌 偵辦 統一 移送 士林 地檢署 檢警 偵辦 重點 在 於 主辦 單位 辦 這個 活動時 到底 有 沒 有 做好 安全 上 的 防備 據 了解 主辦 人 表示 彩色 派 對 今年 是 第四次 舉辦 去年 也 才 在 高雄 西子 灣辦過 不 知道 為何 這次 會 造成 這麼 嚴重 的 意外 但 警方 對 這樣 的 說法 保留 態度 而 士林 地檢署 今天 凌晨 也 已經 由 名檢察官 連夜 到 八仙 樂園 了解 情況 並 扣押 噴發 氣體 和 粉塵 的 鋼瓶 據 了解 檢方 人員到 現場 後 發現 舞台 設在 游泳池 中間 其實 等 於 是 半密閉 空間 而且 地上 四散 菸 蒂 和 打火 機 不 排除 是 有人 不慎 點 菸 引燃 造成 這次 的 意外 而 外界 關注 昨晚 現場 到底 有 沒 有 在 舞台 上 噴火 據 了解 昨晚 器材 中並 沒有 火焰 效果器 檢警 初步 研判 辦 趴用 的 粉彩 主要 成分 為 玉米粉 在 一定 的 空間 一定 的 濃度 遇到 靜電 或 火花 不用 高溫 就 很 容易 引發 閃燃 現象 可能 是 大量 接觸 熱能 像是 舞台 光源 器材 電流 點 菸 等 引發 爆炸 但 詳細 起火 原因 和 起火 點 還是 要 經由 火場 鑑識 釐 清 檢警 近期 將陸續 傳喚 傷者 並調 閱監 視錄 影畫面 了解 事發 經過'

In [99]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
X

<147x12245 sparse matrix of type '<class 'numpy.int64'>'
	with 22679 stored elements in Compressed Sparse Row format>

In [101]:
#vectorizer.get_feature_names()

In [102]:
from sklearn.metrics.pairwise import cosine_similarity
cs = cosine_similarity(X)
cs

array([[ 1.        ,  0.16031292,  0.00604056, ...,  0.25165992,
         0.0303501 ,  0.02004996],
       [ 0.16031292,  1.        ,  0.02020511, ...,  0.1933817 ,
         0.03806935,  0.02235509],
       [ 0.00604056,  0.02020511,  1.        , ...,  0.01585904,
         0.        ,  0.        ],
       ..., 
       [ 0.25165992,  0.1933817 ,  0.01585904, ...,  1.        ,
         0.03984095,  0.05263968],
       [ 0.0303501 ,  0.03806935,  0.        , ...,  0.03984095,
         1.        ,  0.31318412],
       [ 0.02004996,  0.02235509,  0.        , ...,  0.05263968,
         0.31318412,  1.        ]])

In [108]:
from sklearn import cluster
c = cluster.KMeans(n_clusters = 4, init='k-means++')

In [109]:
grp = c.fit_predict(cs)

In [116]:
import numpy as np
titles_ary = np.array(titles)
#titles_ary[grp]
#grp  == 2
for t in titles_ary[grp  == 3]:
    print(t)

蔡依林淚奪金曲 錦榮傳訊恭喜
陳奕迅、張惠妹稱王封后  蔡依林抱回最大獎
陳奕迅、莫文蔚伴侶均不知阿娜答金曲獲獎
金曲26／蔡依林擒３獎大勝　淚崩再挺婚姻平權
金曲26／張惠妹奪歌后卻失落　要世界感受彩虹力量
金曲26／蔡依林淚奪最佳專輯＋完整得獎名單
僅次Jolin！徐佳瑩入圍6獎全槓被封遺珠
金曲最風光！蔡依林紅毯全勝又獲3獎成大贏家
張惠妹3度封后  想破江蕙紀錄
金曲26／陳奕迅稱王謝台灣　張惠妹封后秒噴淚
蔡依林呸大贏家  金曲最佳專輯獎
陳奕迅二度打敗歌神  金曲歌王好嗨
金曲獎完整得獎名單！阿妹封后 陳奕迅稱王
第26屆金曲獎 陳奕迅奪歌王、阿妹封歌后
金曲最佳國語專輯：呸
《金曲26》2015金曲獎得獎名單 線上直播懶人包
